In [ ]:
from constants import Status
from configClasses.baseConfig import BaseConfig
from taskOperator import Operator
from dataHandler import DataHandler

# Configuration Handling
Read in a config file and explore the config object's attributes and methods to obtain parameters. 

In [ ]:
config = BaseConfig.read_ini_config("../../templates/dummy.config")

In [ ]:
# print the contents of the config object
print(config.__dict__)

In [ ]:
# grab the attribute config.jobdb_dict, equivalent to a section in the INI config file
print(config.get_attribute("jobdb_dict"))
# attempt to grab the attribute config.hello but it doesn't exist
print(config.get_attribute("hello","Nope doesn't exist"))

# grab a parameter from the config.jobdb_dict attribute 
print(config.get_parameter("jobdb_dict", "db_name", "Nope doesn't exist"))

# Prepare the task operator object
Using the "dummy" mode, initiate the task operator object which calls the specific strategies used to perform each task. If the input mode is changed, different strategies will be assigned to the task_operator object. 

In [ ]:
task_operator = Operator("dummy")

In [ ]:
print(task_operator.__dict__)

# Prepare the Data Handler object
This is usually done within a context management line (via "with DataHandler(mode, config) as data_handler:") but we are demonstrating the object here so won't use that functionality. 

In [ ]:
data_handler = DataHandler("dummy", config)

In [ ]:
print(data_handler.__dict__)
print(data_handler._strategy.data) # no data has been loaded so will return None

In [ ]:
# the __enter__() method is used to load data into the object and prepare it to be analyzed; for example looped over and printed
data_handler.__enter__()
for key, value in data_handler._strategy.data.items():
    print(key, value)

In [ ]:
# run the get_jobs() method to get an iterator containing the jobs that match the Status
jobs = data_handler.get_jobs(Status.INCOMPLETE)
for job in jobs:
    print(job.job_id, job.status)

In [ ]:
jobs = data_handler.get_jobs(Status.INCOMPLETE)
separator = "#"*80+"\n"
for job in jobs:
    print(separator,job)
    retcode, updates = task_operator.execute(job, config)
    data_handler.update_job(job, updates)
    print(data_handler._strategy.data[job.job_id])


In [ ]:
for key, value in data_handler._strategy.data.items():
    print(key, value['status'])

### Rerun cells 22 and 23 to see how `job.status` changes over multiple iterations of the task Operator executing some function on the job. 